In [121]:
#!pip install lxml

In [122]:
# --- 0. Environment Setup & Offline Preparation ---

# Standard Imports
import os
import glob
import re
import pandas as pd
import lxml.etree as etree
from lxml.etree import _Element as Element # Type hinting for lxml.etree.Element
import collections # For deque in parenthesis removal
import fitz # PyMuPDF for PDF processing
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.modeling_utils import PreTrainedModel
from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.utils.quantization_config import BitsAndBytesConfig
from transformers.training_args import TrainingArguments
from trl import SFTTrainer
import torch
from datasets import Dataset # Hugging Face datasets library
import kagglehub
import spacy
import json

# Set device for PyTorch
device = "cuda" if torch and torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cpu


In [123]:
# Uncomment and run this line once to download the model
#!python -m spacy download en_core_web_sm

In [124]:
# Define constants for file paths and model configurations
BASE_INPUT_DIR = './kaggle/input/make-data-count-finding-data-references'
ARTICLE_TRAIN_DIR = os.path.join(BASE_INPUT_DIR, 'train')
ARTICLE_TEST_DIR = os.path.join(BASE_INPUT_DIR, 'test')

# Define directories for articles in train and test sets
LABELED_TRAINING_DATA_CSV_PATH = os.path.join(BASE_INPUT_DIR, 'train_labels.csv')

# Define the base model path
QWEN_BASE_MODEL_PATH = kagglehub.model_download("qwen-lm/qwen-3/transformers/0.6b")

# Output directory for the fine-tuned model and results
BASE_OUTPUT_DIR = "./kaggle/working"
FINE_TUNED_MODEL_OUTPUT_DIR = os.path.join(BASE_OUTPUT_DIR, "qwen_finetuned_dataset_classifier")
FINAL_RESULTS_CSV_PATH = os.path.join(BASE_OUTPUT_DIR, "article_dataset_classification.csv")

# Load a spaCy model (e.g., 'en_core_web_sm')
# python -m spacy download en_core_web_sm 
NLP_SPACY = spacy.load("en_core_web_sm")


## Data Extraction

### Common dataset identifiers

In [125]:
# --- 2. Information Extraction (IE) - Dataset Identification ---
NON_STD_UNICODE_DASHES = re.compile(r'[\u2010\u2011\u2012\u2013\u2014]')
def clean_text(text: str) -> str:
    """
    Clean the input text by removing non-standard unicode dashes and extra whitespace.
    
    Args:
        text (str): The text to clean.
        
    Returns:
        str: The cleaned text.
    """
    if not text:
        return ""
    # Replace all non-standard unicode dashes with '-'
    text = text.replace('\u200b', '').replace('-\n', '-').replace('_\n', '_').replace('/\n', '/')
    text = NON_STD_UNICODE_DASHES.sub('-', text)
    # Remove extra whitespace
    return re.sub(r'\s+', ' ', text).strip()

# Regex patterns for common dataset identifiers
# DOI_PATTERN = r'10\.\d{4,5}/[-._;()/:A-Za-z0-9\u002D\u2010\u2011\u2012\u2013\u2014\u2015]+'	DOI_PATTERN
# DOI_PATTERN = r'10\.\s?\d{4,5}\/[-._()<>;\/:A-Za-z0-9]+\s?(?:(?![A-Z]+)(?!\d{1,3}\.))+[-._()<>;\/:A-Za-z0-9]+'
#DOI_PATTERN = r'\bhttps://doi.org/10\.\d{4,5}\/[-._\/:A-Za-z0-9]+'
DOI_PATTERN = r'\b10\.\d{4,5}\/[-._\/:A-Za-z0-9]+'
EPI_PATTERN = r'\bEPI[-_A-Z0-9]{2,}'
SAM_PATTERN = r'\bSAMN[0-9]{2,}'          # SAMN07159041
IPR_PATTERN = r'\bIPR[0-9]{2,}'
CHE_PATTERN = r'\bCHEMBL[0-9]{2,}'
PRJ_PATTERN = r'\bPRJ[A-Z0-9]{2,}'
E_G_PATTERN = r'\bE-[A-Z]{4}-[0-9]{2,}'   # E-GEOD-19722 or E-PROT-100
ENS_PATTERN = r'\bENS[A-Z]{4}[0-9]{2,}'
CVC_PATTERN = r'\bCVCL_[A-Z0-9]{2,}'
EMP_PATTERN = r'\bEMPIAR-[0-9]{2,}'
PXD_PATTERN = r'\bPXD[0-9]{2,}'
HPA_PATTERN = r'\bHPA[0-9]{2,}'
SRR_PATTERN = r'\bSRR[0-9]{2,}'
GSE_PATTERN = r'\b(GSE|GSM|GDS|GPL)\d{4,6}\b' # Example for GEO accession numbers (e.g., GSE12345, GSM12345)
GNB_PATTERN = r'\b[A-Z]{1,2}\d{5,6}\b' # GenBank accession numbers (e.g., AB123456, AF000001)
CAB_PATTERN = r'\bCAB[0-9]{2,}'

# Combine all patterns into a list
DATASET_ID_PATTERNS = [
    DOI_PATTERN,
    EPI_PATTERN,
    SAM_PATTERN,
    IPR_PATTERN,
    CHE_PATTERN,
    PRJ_PATTERN,
    E_G_PATTERN,
    ENS_PATTERN,
    CVC_PATTERN,
    EMP_PATTERN,
    PXD_PATTERN,
    HPA_PATTERN,
    SRR_PATTERN,
    GSE_PATTERN,
    GNB_PATTERN,
    CAB_PATTERN,
]

# Compile all patterns for efficiency
COMPILED_DATASET_ID_REGEXES = [re.compile(p) for p in DATASET_ID_PATTERNS]

# Data related keywords to look for in the text
# These keywords help to ensure that the text is relevant to datasets
DATA_RELATED_KEYWORDS = ['data release', 'data associated', 'data availability', 'data access', 'download', 'program data', 'the data', 'dataset', 'database', 'repository', 'data source', 'data access', 'archive', 'arch.', 'digital']

def is_text_data_related(text: str) -> bool:
    if not text:
        return False
    
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in DATA_RELATED_KEYWORDS)

def text_has_dataset_id(text: str) -> bool:
    """
    Check if the given text contains any dataset identifier.
    
    Args:
        text (str): The text to check for dataset identifiers.
        
    Returns:
        bool: True if any dataset identifier is found, False otherwise.
    """

    occurrences_with_context: list[str] = []
    for regex in COMPILED_DATASET_ID_REGEXES:
        if regex.search(text):
            text_lower = text.lower()
            # Check for specific keywords in the text
            if any(keyword in text_lower for keyword in DATA_RELATED_KEYWORDS):
                return True
    return False

def extract_dataset_ids(text: str, context_chars: int = 250) -> str:
    """
    Extract dataset identifiers with context from the given text.
    
    Args:
        text (str): The text to search for dataset identifiers.
        context_chars (int): Number of characters to include before and after the match for context.
        
    Returns:
        list[str]: A list of extracted dataset identifiers with context.
    """
    text = clean_text(text)
    is_small_context = len(text) < context_chars * 2
    dataset_ids: list[str] = []
    occurrences_with_context: list[str] = []
    if is_text_data_related(text):
        for regex in COMPILED_DATASET_ID_REGEXES:
            matches = regex.finditer(text, re.IGNORECASE)
            for match in matches:
                dataset_id = text[match.start() : match.end()]
                if is_small_context:
                    dataset_ids.append(dataset_id)
                else:
                    citation_context = text[max(0, match.start() - context_chars): match.end() + context_chars ]
                    citation_context = citation_context.replace('\n', '').replace('[', '').replace(']', '')
                    citation_context = re.sub(r'\s+', ' ', citation_context).strip()
                    if is_text_data_related(citation_context):
                        occurrences_with_context.append("{" + f'"dataset_ids": {[dataset_id]}, citation_context: "{citation_context}"' + "}")
        if dataset_ids:
            occurrences_with_context.append("{" + f'"dataset_ids": {dataset_ids}, citation_context: "{text}"' + "}")
    
    # If no occurrences found, return an empty string
    # Otherwise, join the occurrences with a specific separator
    if not occurrences_with_context:
        return ""
    return ",".join(occurrences_with_context)

# Use NLP to get sentences from the given text

def get_sentences_from_text(text: str, nlp=NLP_SPACY) -> str:
    if not text:
        return ""
    
    # Replace all non-standard unicode dashes with '-'
    text = clean_text(text)
    text = text.replace('\n', ' ').strip()
    doc_spacy = nlp(text)
    return "\n".join([sent.text for sent in doc_spacy.sents])

### XML Element Extraction

In [126]:

def extract_element_text(element: Element | None) -> str:
    if element is not None:
        # Use itertext() to get all text content from the <p> tag and its descendants
        # and join them into a single string.
        all_text = " ".join(element.itertext(tag=None)).replace('\u200b', '').strip()
        return all_text[:2000]
    else:
        return ""
    
def extract_next_sibling_text(elements: list[Element] | None, sibling_xpath: str) -> str:
    """
    Extracts text from the next sibling of the given XML element.
    
    Args:
        element (Element | None): The XML element whose next sibling's text is to be extracted.
        sibling_xpath (str): The XPath expression to find the next sibling element. (eg. "following-sibling::passage[1]")
        
    Returns:
        str: A string containing the text from the next sibling element, or an empty string if no sibling exists.

    """
    # Check if the provided elements list is None or empty
    if not elements:
        return ""
    
    # Assuming there's only one such element, take the first one found
    # and find the element immediately following based on the given sibling_xpath.
    first_element = elements[0]
    sibling_elements = first_element.xpath(sibling_xpath)

    if not sibling_elements:
        # print("DEBUG: No following <passage> element found.") # Uncomment for debugging
        return ""
    
    next_sibling = sibling_elements[0]
    if next_sibling is None:
        return ""
    
    return extract_element_text(next_sibling)

def extract_elements_text(elements: list[Element] | None, sep: str = " ") -> str:
    elements_text = []
    if elements is None:
        return ""
    
    for element in elements:
        text = extract_element_text(element)
        if text:
            elements_text.append(text)

    return sep.join(elements_text).strip()

def extract_elements_text_from_xpath_list(root: Element | None, xpath_list: list[str], ns: dict[str, str] | None = None) -> str:
    elements_text = ""
    if root is None or not xpath_list:
        return ""
    
    for xpath in xpath_list:
        element = root.find(xpath, namespaces=ns)
        elements_text += extract_element_text(element)
    return elements_text

def extract_text_from_elements_within_element(element: Element | None, child_xpaths: list[str] = [], ns: dict[str, str] | None = None) -> str:
    """
    Extracts text from elements within a given XML element that match the specified tag names.
    
    Args:
        element (Element | None): The XML element to search within.
        tag_names (list[str]): A list of tag names to search for.
        
    Returns:
        str: A string containing the extracted text from the matching elements.
    """
    if element is None:
        return ""
    
    if not child_xpaths:
        # If no child tag names are provided, return the text of the element itself
        return extract_element_text(element)
    
    extracted_text = []
    for xpath in child_xpaths:
        for child in element.findall(xpath, namespaces=ns):
            text = extract_element_text(child)
            if text:
                extracted_text.append(text)
    
    return "|".join(extracted_text)

def extract_data_related_elements_text(elements: list[Element] | None, child_xpaths: list[str] = [], ns: dict[str, str] | None = None) -> list[str]:
    elements_text = []
    if elements is None:
        return elements_text
    
    for element in elements:
        text = extract_dataset_ids(extract_text_from_elements_within_element(element, child_xpaths, ns))
        if text:
            elements_text.append(text)

    return elements_text

def extract_data_related_elements_text_from_xpath_list(root: Element | None, xpath_list: list[str], ns: dict[str, str] | None = None) -> list[str]:
    """
    Extracts text from elements in the XML tree that match the provided XPath expressions.
    
    Args:
        root (Element | None): The root element of the XML tree.
        xpath_list (list[str]): A list of XPath expressions to search for elements.
        
    Returns:
        list[str]: A list of extracted text from the matching elements.
    """
    elements_text = []
    if root is None or not xpath_list:
        return elements_text
    
    for xpath in xpath_list:
        primary_xpath, *child_xpath_text = xpath.split('||')
        child_xpaths = child_xpath_text[0].split(',') if child_xpath_text else []
        elements = root.findall(primary_xpath, namespaces=ns)
        if elements:
            elements_text.extend(extract_data_related_elements_text(elements, child_xpaths, ns))
    return elements_text




### PDF File Extraction

In [127]:

def extract_author_names(full_text: str, nlp=NLP_SPACY) -> str:
    """
    Extracts potential author names from the beginning of a research article's text
    using spaCy's Named Entity Recognition. It attempts to isolate the author section
    and applies heuristics to filter out non-author entities.

    Args:
        full_text (str): The complete text content of the research article,
                         typically extracted from a PDF.

    Returns:
        List[str]: A list of unique strings, each representing a potential author name,
                   sorted alphabetically. Returns an empty list if no authors are found.
    """
    if not full_text or not full_text.strip():
        return ""

    full_text = full_text.replace('1\n,', ',').replace('1,', ',').replace('\u2019', "'")

    # 1. Isolate the potential author section
    # Authors are typically at the very beginning, before the abstract or introduction.
    # We'll search for common section headers to define the end of the author block.
    # Using regex for case-insensitive search and handling various newline/spacing.
    header_patterns = [
        r"\n\s*Abstract\s*\n",
        r"\n\s*Introduction\s*\n",
        r"\n\s*Summary\s*\n",
        r"\n\s*Keywords\s*\n",
        r"\n\s*Graphical Abstract\s*\n",
        r"\n\s*1\.\s*Introduction\s*\n", # Common for numbered sections
        r"\n\s*DOI:\s*\n" # Sometimes DOI appears before abstract
    ]

    author_section_end_index = len(full_text)
    for pattern in header_patterns:
        match = re.search(pattern, full_text, re.IGNORECASE)
        if match:
            # Take text up to the start of the found header
            author_section_end_index = min(author_section_end_index, match.start())
            break
    
    # As a fallback or if no header is found early, limit the search to the first
    # 2500 characters. This prevents processing the entire document for authors.
    author_section_text = full_text[:min(author_section_end_index, 2500)]

    if not author_section_text.strip():
        return ""

    # 2. Process the isolated author section with spaCy
    doc = nlp(author_section_text)

    # 3. Extract PERSON entities and apply initial filtering
    potential_authors: list[str] = []
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            name = ent.text.strip()
            # Basic filtering to reduce false positives:
            # - Exclude very short strings (e.g., single letters, common conjunctions)
            # - Exclude common stop words (e.g., "The", "And")
            # - Exclude all-uppercase strings that might be acronyms (e.g., "WHO", "NASA")
            # - Ensure it contains at least one space (e.g., "John Doe") or is a capitalized
            #   single word that's longer than 2 characters (e.g., "Smith").
            if (len(name) > 1 and
                name.lower() not in nlp.Defaults.stop_words and
                not name.isupper() and
                (' ' in name or (name[0].isupper() and len(name) > 2))):
                
                potential_authors.append(name)

    # 4. Apply more advanced heuristics to filter out non-author names
    # This step is crucial for accuracy and often requires tuning.
    filtered_authors = []
    for author in potential_authors:
        # Heuristic 1: Filter out names that contain common affiliation keywords.
        # This is a simple check; more robust solutions might use spaCy's dependency
        # parsing to check if a PERSON entity is part of an ORG entity.
        affiliation_keywords = ["univ", "observ", "institute", "department", "center", "lab",
                                "hospital", "college", "school", "inc.", "ltd.", "company",
                                "corp.", "group", "foundation", "research"]
        if any(keyword in author.lower() for keyword in affiliation_keywords):
            continue # Skip if it looks like an affiliation

        # Heuristic 2: Filter out names that contain email patterns or ORCID patterns.
        if '@' in author or re.search(r'\b\d{4}-\d{4}-\d{4}-\d{3}[\dX]\b', author):
            continue # Skip if it contains an email or ORCID

        # Heuristic 3: Filter out names that are likely just initials or very short.
        # This is partially covered by initial filtering, but can be refined.
        # E.g., "J. D." might be an author, but "J." alone is unlikely.
        if len(author.split()) == 1 and len(author) <= 2 and author.isupper():
            continue # Skip single-letter or two-letter uppercase (e.g., "JD")

        filtered_authors.append(author)

    # Convert to list and sort for consistent output
    return filtered_authors[0] if filtered_authors else ""

def extract_pdf_doc_text(pdf_doc: fitz.Document)  -> dict[str, str | list[str]]:
    """
    Extracts all text from a PDF document using PyMuPDF.
    
    Args:
        pdf_doc (fitz.Document): The PDF document to extract text from.
        
    Returns:
        str: A JSON string of the article_dict containing specific elements extracted from the PDF.
    """

    # Initialize the article dictionary with empty strings
    article_dict = {
        'title': '',
        'author': '',
        'abstract': '',
        'data_availability': '',
        'other_dataset_citations': []
    }

    # Initialize variables for text extraction
    p1 = None  # Placeholder for the first page text
    other_dataset_citations = set()  # Use a set to avoid duplicates
    for page in pdf_doc:
        # Extract text from the page
        textpage = page.get_textpage()
        if page.number == 0:
            p1_txt = textpage.extractTEXT()
            p1 = get_sentences_from_text(p1_txt)
            p1 = p1[:int(len(p1)/2)]
            article_dict['author'] = extract_author_names(p1_txt, nlp=NLP_SPACY)

        # Extract text from all blocks that have an abstract or dataset id's
        blocks = textpage.extractBLOCKS()
        for block in blocks:
            block_text = get_sentences_from_text(block[4])
            block_text_lower = block_text.lower()
            if page.number == 0 and len(block_text) > 100 and "abstract" in block_text_lower:
                # Add the abstract block text to the article dictionary
                article_dict['abstract'] = block_text
            elif "data availability" in block_text_lower or "data accessibility" in block_text_lower or "acknowledgments" in block_text_lower:
                # Add the data availability block text to the article dictionary
                article_dict['data_availability'] = block_text
            else:
                context_chars = min(250, len(block_text))  # Use a minimum
                dataset_ids_found = extract_dataset_ids(block_text, context_chars)  # Extract dataset IDs from the block text
                if dataset_ids_found:
                    if len(article_dict['data_availability']) > 0 and len(article_dict['data_availability']) < 25:
                        # If data availability text is only a few characters, append the next block text to it
                        # This is a heuristic to ensure that we capture relevant dataset IDs
                        article_dict['data_availability'] = block_text
                    else:
                        # Append the dataset IDs found in the block to the other_dataset_citations
                        other_dataset_citations.add(dataset_ids_found)

    article_dict['other_dataset_citations'] = list(other_dataset_citations) if other_dataset_citations else []
    
    # If an abstract was not found, use the first page text as the abstract
    if not article_dict['abstract'] and p1:
        article_dict['abstract'] = p1

    # Return the article dictionary as a JSON string
    return article_dict


### XML File Extraction

In [128]:

def extract_xml_text_jats(root: Element) -> dict[str, str | list[str]]:
    # Find the title, abstract, and data availablity info for Journal Archiving and Interchange DTD (JATS)
    # The ".//" ensures it searches anywhere in the document, not just direct children of root.
    ns = None  # No namespaces for JATS

    xpath_title = ".//article-title"
    xpath_authors_1 = ".//contrib-group/contrib[@contrib-type='author']/name"
    xpath_authors_2 = ".//biblstruct/analytic/author[@role='corresp']/persname"
    author = extract_element_text(root.find(xpath_authors_1, namespaces=ns))
    if not author:
        author = extract_element_text(root.find(xpath_authors_2, namespaces=ns))
    xpath_abstract = ".//abstract"
    xpath_data_avails = [".//notes[@notes-type='data-availability']", ".//sec[@sec-type='data-availability']"]
    xpath_citations = [".//element-citation||.article-title,.source,.pub-id", ".//mixed-citation"]  # List of XPath expressions for citations

    return {
        'title': extract_element_text(root.find(xpath_title, ns)),
        'author': author,
        'abstract': extract_element_text(root.find(xpath_abstract, ns)),
        'data_availability': extract_elements_text_from_xpath_list(root, xpath_data_avails, ns=ns),
        'other_dataset_citations': extract_data_related_elements_text_from_xpath_list(root, xpath_citations, ns=ns),
    }

def extract_xml_text_tei(root: Element) -> dict[str, str | list[str]]:
    # Find the title, abstract, and data availability info for Text Encoding Initiative (TEI)
    # Set the namespace for TEI
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

    xpath_title = ".//tei:title"
    xpath_authors = ".//tei:sourcedesc/tei:biblstruct/tei:analytic/tei:author/tei:persname"
    xpath_abstract = ".//tei:abstract"
    xpath_data_avail = "" #".//tei:biblstruct"
    xpath_citations = [".//tei:biblstruct||.//tei:title,.//tei:idno,.//tei:notes"]  # List of XPath expressions for citations
        
    return {
        'title': extract_element_text(root.find(xpath_title, namespaces=ns)),
        'author': extract_element_text(root.find(xpath_authors, namespaces=ns)),
        'abstract': extract_element_text(root.find(xpath_abstract, namespaces=ns)),
        'data_availability': xpath_data_avail,  # No direct extraction for TEI data_availability
        'other_dataset_citations': extract_data_related_elements_text_from_xpath_list(root, xpath_citations, ns=ns),
    }

def extract_xml_text_wiley(root: Element) -> dict[str, str | list[str]]:
    # Find the title, abstract, and data availability info for Wiley XML format
    # Set the namespace for Wiley
    ns = {'ns': 'http://www.wiley.com/namespaces/wiley'}

    xpath_title = ".//ns:publicationMeta[@level='part']/ns:titleGroup"    #<publicationMeta level="part"><titleGroup><title type="main">
    xpath_authors = ".//selfCitationGroup/citation[@type='self']/author"
    xpath_abstract = ".//ns:abstract[@type='main']"  #<abstract type="main"
    xpath_data_avail = ".//ns:section[@type='dataAvailability']"  #<section numbered="no" type="dataAvailability"
    xpath_citations = [".//ns:citation||.//ns:articleTitle,.//ns:journalTitle,.//ns:url"]  # List of XPath expressions for citations
        
    return {
        'title': extract_elements_text(root.findall(xpath_title, namespaces=ns)),
        'author': extract_element_text(root.find(xpath_authors, namespaces=ns)),
        'abstract': extract_element_text(root.find(xpath_abstract, namespaces=ns)),
        'data_availability': extract_element_text(root.find(xpath_data_avail, namespaces=ns)),
        'other_dataset_citations': extract_data_related_elements_text_from_xpath_list(root, xpath_citations, ns=ns),
    }

def extract_xml_text_biorxiv(root: Element) -> dict[str, str | list[str]]:
    # Find the title, abstract, and data availability info for BioRxiv XML format
    # Set the namespace for BioRxiv
    ns = {'biorxiv': 'http://www.biorxiv.org'}

    xpath_title = ".//biorxiv:title"
    xpath_authors = ".//biorxiv:contrib[@contrib-type='author']/biorxiv:name"
    xpath_abstract = ".//biorxiv:abstract"
    xpath_data_avail = ".//biorxiv:sec[@sec-type='data-availability']"  #<sec sec-type="data-availability"
    xpath_citations = [".//biorxiv:biblio||.//biorxiv:title,.//biorxiv:source,.//biorxiv:pub-id"]  # List of XPath expressions for citations
        
    return {
        'title': extract_element_text(root.find(xpath_title, namespaces=ns)),
        'author': extract_element_text(root.find(xpath_authors, namespaces=ns)),
        'abstract': extract_element_text(root.find(xpath_abstract, namespaces=ns)),
        'data_availability': extract_element_text(root.find(xpath_data_avail, namespaces=ns)),
        'other_dataset_citations': extract_data_related_elements_text_from_xpath_list(root, xpath_citations, ns=ns),
    }

def extract_xml_text_bioc(root: Element) -> dict[str, str | list[str]]:
    # Find the title, abstract, and data availability info for BioC-API XML format
    ns = None  # No namespaces for BioC

    xpath_title = "string(.//passage[infon[@key='section_type' and text()='TITLE']]/text)"
    xpath_authors = "string(.//infon[@key='name_0'] | .//infon[@key='name_1'])"
    xpath_abstract = "string(.//passage[infon[@key='section_type' and text()='ABSTRACT']]/text)"
    xpath_data_avail = ".//passage[text[text()='DATA ACCESSIBILITY:']]"
    xpath_data_avail_sibling = "following-sibling::passage[1]"
    xpath_citations = []
        
    return {
        'title': root.xpath(xpath_title, namespaces=ns),
        'author': root.xpath(xpath_authors, namespaces=ns).strip().replace('surname:', '').replace(';given-names:', ' '),
        'abstract': root.xpath(xpath_abstract, namespaces=ns)[:2000],  # Limit to 2000 characters
        'data_availability': extract_next_sibling_text(root.xpath(xpath_data_avail, namespaces=ns), xpath_data_avail_sibling),
        'other_dataset_citations': xpath_citations,
    }

def extract_xml_text_taxonx(root: Element) -> dict[str, str | list[str]]:
    # Find the title, abstract, and data availability info for TaxonX format
    ns = None  # No namespaces for Taxonomic Treatment Publishing DTD

    xpath_title = "string(.//article-meta/title-group/article-title)"
    xpath_authors = ""
    xpath_abstract = "string(.//article-meta/abstract)"
    xpath_data_avail = ""
    xpath_citations = []
        
    return {
        'title': root.xpath(xpath_title, namespaces=ns),
        'author': xpath_authors,
        'abstract': root.xpath(xpath_abstract, namespaces=ns)[:2000],  # Limit to 2000 characters
        'data_availability': xpath_data_avail,  # No direct extraction for TaxonX data_availability
        'other_dataset_citations': xpath_citations,
    }


## File Processing

In [129]:

# Dictionary mapping XML types to their respective extraction functions
XML_TYPE_EXTRACTORS = {
    'jats': extract_xml_text_jats,
    'tei': extract_xml_text_tei,
    'wiley': extract_xml_text_wiley,
    'bioc': extract_xml_text_bioc,
    'taxonx': extract_xml_text_taxonx,
}

# --- Data Loading ---
def get_file_extension(file_path: str) -> str:
    """
    Returns the file extension of the given file path.
    
    Args:
        file_path (str): The path to the file.
        
    Returns:
        str: The file extension, or an empty string if no extension is found.
    """
    _, ext = os.path.splitext(file_path)
    return ext.lower() if ext else ""

def read_first_line_of_xml(file_path: str) -> str | None:
    """
    Reads and returns the first line of an XML file.

    Args:
        file_path (str): The path to the XML file.

    Returns:
        str | None: The first line of the file, stripped of leading/trailing whitespace,
                    or None if the file cannot be read or is empty.
    """
    if not file_path and not os.path.exists(file_path):
        return None
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            first_line = f.readline().replace('<?xml version="1.0" encoding="UTF-8"?>', '').replace('<?xml version="1.0" encoding="UTF-8" standalone="yes"?>', '').strip()
            # If the first line is empty, read the next line
            if not first_line:
                first_line = f.readline()
            return first_line.strip()[:90] if first_line else None
    except UnicodeDecodeError:
        try:
            with open(file_path, 'r', encoding='iso-8859-1') as f:
                first_line = f.readline().replace('<?xml version="1.0" encoding="UTF-8"?>', '').replace('<?xml version="1.0" encoding="UTF-8" standalone="yes"?>', '').strip()
                if not first_line:
                    first_line = f.readline()
                return first_line.strip()[:90] if first_line else None
        except Exception as e:
            return None
    except Exception as e:
        print(f"Error reading file '{file_path}': {e}")
        return None
    
def identify_xml_type(first_line: str) -> str:
    """
    Identifies the XML type based on the first line of the XML file.
    
    Args:
        file_path (str): The path to the XML file.
        
    Returns:
        str: The identified XML type ('jats', 'tei', 'wiley', 'bioc', or 'unknown').
    """
    if not first_line:
        return "unknown"
    first_line_lower = first_line.lower()
    # Check for specific patterns in the first line
    if 'journal archiving and interchange dtd' in first_line_lower:
        return "jats"
    elif 'xmlns="http://www.tei-c.org/ns/1.0"' in first_line_lower:
        return "tei"
    elif 'xmlns="http://www.wiley.com/namespaces/wiley"' in first_line_lower:
        return "wiley"
    elif 'bioc.dtd' in first_line_lower or 'bioc-api' in first_line_lower:
        return "bioc"
    elif 'taxonomic treatment publishing dtd' in first_line_lower:
        return "taxonx"
    
    return "unknown"    

def get_xml_type(file_path: str) -> str:
    """
    Determines the XML type of a file based on its first line.
    
    Args:
        file_path (str): The path to the XML file.
        
    Returns:
        str: The identified XML type ('jats', 'tei', 'wiley', 'bioc', 'taxonx', or 'unknown').
    """
    first_line = ""
    if ".xml" == get_file_extension(file_path):
        # If the file is an XML file, read the first line and identify the type
        first_line = read_first_line_of_xml(file_path)
    return identify_xml_type(first_line) if first_line else "unknown"

def load_file_paths(dataset_type_dir: str) -> pd.DataFrame: 
    pdf_path = os.path.join(dataset_type_dir, 'PDF')
    xml_path = os.path.join(dataset_type_dir, 'XML')
    dataset_type = os.path.basename(dataset_type_dir)
    pdf_files = [f for f in os.listdir(pdf_path) if f.endswith('.pdf')]
    xml_files = [f for f in os.listdir(xml_path) if f.endswith('.xml')]
    df_pdf = pd.DataFrame({
        'article_id': [f.replace('.pdf', '') for f in pdf_files],
        'pdf_file_path': [os.path.join(pdf_path, f) for f in pdf_files]
    })
    df_xml = pd.DataFrame({
        'article_id': [f.replace('.xml', '') for f in xml_files],
        'xml_file_path': [os.path.join(xml_path, f) for f in xml_files]
    })
    merge_df = pd.merge(df_pdf, df_xml, on='article_id', how='outer', suffixes=('_pdf', '_xml'), validate="one_to_many")
    merge_df['dataset_type'] = dataset_type
    return merge_df

def extract_pdf_text(file_path: str, xml_type: str | None = None) -> dict[str, str | list[str]]:
    """Extracts all text from a PDF file using PyMuPDF."""
    article_dict = {}
    if file_path and os.path.exists(file_path):
        try:
            with fitz.open(file_path) as doc:
                article_dict = extract_pdf_doc_text(doc)  # Extract text from the PDF document
        except Exception as e:
            print(f"Error reading PDF {file_path}: {e}")
    else:
        print(f"PDF file not found: {file_path}")
    
    return article_dict

def extract_xml_text(file_path: str, xml_type: str) -> dict[str, str | list[str]]:
    """Reads and extracts text from an XML file based on the specified XML type.
    Args:
        file_path (str): The path to the XML file.
        xml_type (str): The type of XML format (e.g., 'jats', 'tei', 'wiley', 'bioc', 'taxonx').
    Returns:
        dict: A dictionary containing the extracted text from the XML file.
    """
    # Initialize the article dictionary
    article_dict = {}
    if file_path and os.path.exists(file_path):
        # Disable external entity resolution for security
        parser = etree.XMLParser(resolve_entities=False, no_network=True)
        try:
            tree = etree.parse(file_path, parser)
            root = tree.getroot()
            # Use the appropriate extraction function based on the xml_type
            extract_function = XML_TYPE_EXTRACTORS.get(xml_type, extract_xml_text_jats)  
            article_dict = extract_function(root)
        except Exception as e:
            print(f"Error reading XML {file_path}: {e}")
    else:
        print(f"XML file not found: {file_path}")    
    return article_dict

def process_unsupported_file(file_path: str, xml_type: str | None = None) -> str:
    return f"Unsupported file type for: {file_path}"

# Dictionary mapping file extensions to loading functions
FILE_EXTRACTORS = {
    '.xml': extract_xml_text,
    '.pdf': extract_pdf_text,
}

def extract_article_text(file_path: str) -> str:
    """
    Loads text content from a single article file (PDF or XML).
    Returns the text content of the given file.
    """
    text_content = ""

    # Get the file extension (e.g., '.xml', '.pdf')
    file_extension = get_file_extension(file_path)

    # Get the XML type if the file is an XML file
    xml_type = get_xml_type(file_path)

    # Get the appropriate function from the dictionary,
    # or fall back to a default 'unsupported' function if not found.
    extract_function = FILE_EXTRACTORS.get(file_extension, process_unsupported_file)

    # Call the selected function
    article_dict = extract_function(file_path, xml_type=xml_type)
    text_content = json.dumps(article_dict, separators=(',', ':'))
    print(f"Extracted text from {file_path}. Length: {len(text_content)} characters, xml_type: {xml_type}")

    return text_content


In [130]:
# Test extracting text from various PDF and XML files
pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'PDF', '10.1002_2017jc013030.pdf')
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'XML', '10.1002_2017jc013030.xml')
# pdf_file_path = os.path.join(ARTICLE_TEST_DIR, 'PDF', '10.1002_ecs2.1280.pdf')
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'PDF', '10.1017_rdc.2022.19.pdf')
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'PDF', '10.1017_s0007123423000601.pdf')
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'PDF', '10.3389_fcimb.2024.1292467.pdf')
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'PDF', '10.1002_esp.5058.pdf') # This one is big
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'PDF', '10.1002_esp.5059.pdf') # This one is big
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'PDF', '10.1002_ece3.4466.pdf') # dryad
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'XML', '10.1002_ece3.4466.xml') # dryad
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'XML', '10.1002_mp.14424.xml')
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'XML', '10.1007_s00259-022-06053-8.xml')    # jats
pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'XML', '10.1007_s00382-022-06361-7.xml')    # tei
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'XML', '10.1111_1365-2435.13431.xml')       # wiley
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'XML', '10.1111_mec.16977.xml')             # bioc
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'XML', '10.3897_zoologia.35.e23481.xml')      # taxonx
# pdf_file_path = os.path.join(ARTICLE_TRAIN_DIR, 'XML', '10.1002_ece3.6144.xml')               # jats
article_text = extract_article_text(pdf_file_path)
display(len(article_text))
article_text

Extracted text from ./kaggle/input/make-data-count-finding-data-references\train\XML\10.1007_s00382-022-06361-7.xml. Length: 1888 characters, xml_type: tei


1888

'{"title":"Winter wet-dry weather patterns driving atmospheric rivers and Santa Ana winds provide evidence for increasing wildfire hazard in California","author":"Kristen Guirguis","abstract":"Floods caused by atmospheric rivers and wildfires fanned by Santa Ana winds are common occurrences in California with devastating societal impacts. In this work, we show that winter weather variability in California, including the occurrence of extreme and impactful events, is linked to four atmospheric circulation regimes over the North Pacific Ocean previously named and identified as the \\"NP4 modes\\". These modes come in and out of phase with each other during the season, resulting in distinct weather patterns that recur throughout the historical record. Some phase combinations favor atmospheric river landfalls and extreme daily or multi-day precipitation, while other phase combinations favor anomalously hot weather and drying Santa Ana wind conditions over Southern California. This historic

## Pre-processing

In [134]:
# Load the labeled training data CSV file
print(f"Loading labeled training data from: {LABELED_TRAINING_DATA_CSV_PATH}")
train_labels_df = pd.read_csv(LABELED_TRAINING_DATA_CSV_PATH)

print(f"Training labels shape: {train_labels_df.shape}")
display(train_labels_df.head())


Loading labeled training data from: ./kaggle/input/make-data-count-finding-data-references\train_labels.csv
Training labels shape: (1028, 3)


,article_id,dataset_id,type
0,10.1002_2017jc013030,https://doi.org/10.17882/49388,Primary
1,10.1002_anie.201916483,Missing,Missing
2,10.1002_anie.202005531,Missing,Missing
3,10.1002_anie.202007717,Missing,Missing
4,10.1002_chem.201902131,Missing,Missing


In [135]:
# Group training data by article_id to get all datasets for each article
# This creates a dictionary where keys are article_ids and values are lists of dataset dicts
grouped_training_data = {}
for article_id, group_df in train_labels_df.groupby('article_id'):
    grouped_training_data[article_id] = group_df[['dataset_id', 'type']].to_dict('records')

# Example usage of grouped_training_data
print(f"Example grouped training data for article_id '10.1002_2017jc013030': {grouped_training_data['10.1002_2017jc013030']}")


Example grouped training data for article_id '10.1002_2017jc013030': [{'dataset_id': 'https://doi.org/10.17882/49388', 'type': 'Primary'}]


In [195]:
# Set the base file dir for the articles to be processed
base_file_dir = ARTICLE_TEST_DIR \
    if os.getenv('KAGGLE_IS_COMPETITION_RERUN') \
    else ARTICLE_TRAIN_DIR

# Just for testing, always set to the ARTICLE_TEST_DIR
base_file_dir = ARTICLE_TEST_DIR

In [158]:
# Load file paths for training and testing datasets
file_paths_df = load_file_paths(base_file_dir)
file_paths_df['xml_file_path'] = file_paths_df['xml_file_path'].fillna('')

# Merge the file paths with the grouped_training_data
file_paths_df['dataset_info'] = file_paths_df['article_id'].map(grouped_training_data)

# Get the xml type for each file based on the first line of the XML file
file_paths_df['xml_type'] = file_paths_df['xml_file_path'].apply(get_xml_type)

print(f"Files paths shape: {file_paths_df.shape}")
display(file_paths_df.sample(3))


Files paths shape: (30, 6)


,article_id,pdf_file_path,xml_file_path,dataset_type,dataset_info,xml_type
17,10.1002_ece3.6784,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test,"[{'dataset_id': 'Missing', 'type': 'Missing'}]",jats
5,10.1002_chem.201903120,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test,"[{'dataset_id': 'Missing', 'type': 'Missing'}]",jats
0,10.1002_2017jc013030,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test,[{'dataset_id': 'https://doi.org/10.17882/4938...,tei


In [173]:
# Load the PDF text content for each article
file_paths_df['pdf_text'] = file_paths_df['pdf_file_path'].apply(lambda x: extract_article_text(x) if x else "")
# Load the XML text content for each article
file_paths_df['xml_text'] = file_paths_df['xml_file_path'].apply(lambda x: extract_article_text(x) if x else "")
# Fill NaN values in the 'xml_type' and 'xml_text' columns with empty strings
file_paths_df['xml_file_path'] = file_paths_df['xml_file_path'].fillna('')
file_paths_df['xml_type'] = file_paths_df['xml_type'].fillna('')
file_paths_df['xml_text'] = file_paths_df['xml_text'].fillna('')
# Display the first few rows of the training file paths DataFrame
print(f"File paths DataFrame shape: {file_paths_df.shape}")
display(file_paths_df.head(3))
# Save the file paths DataFrame to CSV files
file_paths_df.to_csv(os.path.join(BASE_OUTPUT_DIR, 'file_paths.csv'), index=False)


Extracted text from ./kaggle/input/make-data-count-finding-data-references\test\PDF\10.1002_2017jc013030.pdf. Length: 6686 characters, xml_type: unknown
Extracted text from ./kaggle/input/make-data-count-finding-data-references\test\PDF\10.1002_anie.201916483.pdf. Length: 1336 characters, xml_type: unknown
Extracted text from ./kaggle/input/make-data-count-finding-data-references\test\PDF\10.1002_anie.202005531.pdf. Length: 1622 characters, xml_type: unknown
Extracted text from ./kaggle/input/make-data-count-finding-data-references\test\PDF\10.1002_anie.202007717.pdf. Length: 1710 characters, xml_type: unknown
Extracted text from ./kaggle/input/make-data-count-finding-data-references\test\PDF\10.1002_chem.201902131.pdf. Length: 648 characters, xml_type: unknown
Extracted text from ./kaggle/input/make-data-count-finding-data-references\test\PDF\10.1002_chem.201903120.pdf. Length: 872 characters, xml_type: unknown
Extracted text from ./kaggle/input/make-data-count-finding-data-references

,article_id,pdf_file_path,xml_file_path,dataset_type,dataset_info,xml_type,pdf_text,xml_text
0,10.1002_2017jc013030,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test,[{'dataset_id': 'https://doi.org/10.17882/4938...,tei,"{""title"":"""",""author"":""Marie Barbieux"",""abstrac...","{""title"":""Assessing the variability in the rel..."
1,10.1002_anie.201916483,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test,"[{'dataset_id': 'Missing', 'type': 'Missing'}]",jats,"{""title"":"""",""author"":""Daniel Werner"",""abstract...","{""title"":""Effective and Reversible Carbon Diox..."
2,10.1002_anie.202005531,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test,"[{'dataset_id': 'Missing', 'type': 'Missing'}]",bioc,"{""title"":"""",""author"":""Dong Wang"",""abstract"":""A...","{""title"":""Trifluoromethyl Sulfoxides: Reagents..."


## Define the Qwen evaluation model class

In [196]:
# --- QwenModelEval Class ---
# kagglehub.model_download("qwen-lm/qwen-3/transformers/0.6b")
#max_new_tokens=32768
class QwenModelEval:
    def __init__(self, model_name, sys_prompt, enable_thinking=True, max_new_tokens=1024, max_input_length=8200):
        print(f"Loading Qwen model and tokenizer from: {model_name}")
        self.model_name = model_name
        self.sys_prompt = sys_prompt
        self.enable_thinking = enable_thinking  # Enable or disable thinking mode
        self.max_new_tokens = max_new_tokens  # Set the maximum number of new tokens to generate
        self.max_input_length = max_input_length  # Set the maximum input length for the model
        # Load the tokenizer and model
        # Using trust_remote_code=True to allow custom model code execution
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto", trust_remote_code=True)
        self.model.eval() # Set the model to evaluation mode here.

    def generate_response(self, user_input):  
        inputs = self._get_inputs(user_input)
        # Disable gradient calculation during inference
        # Generate the response using the model
        with torch.no_grad(): 
            generated_ids = self.model.generate(
                **inputs,
                max_new_tokens=self.max_new_tokens,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.convert_tokens_to_ids("<|im_end|>"),
                # do_sample=False, # Use greedy decoding (fastest)
                # num_beams=1,     # Do not use beam search (fastest)
                # temperature=0.0, # Make output deterministic (if do_sample=False, this has no effect)                
                temperature=0.6 if self.enable_thinking else 0.7,
                top_p=0.95 if self.enable_thinking else 0.8,
                top_k=20,
                min_p=0
            )
        # Parse the response and thinking content
        return self._parse_response(inputs, generated_ids)

    def _get_inputs(self, user_input):
        """Prepare the input for the model based on user input."""
        # Trim the user input to a maximum length for better performance
        user_input = user_input[:self.max_input_length]  # Limit input length to 4096 characters
        print(f"Preparing input with length: {len(user_input)}")
        # Create the messages for the chat template
        messages = [
            {"role": "system", "content": self.sys_prompt},
            {"role": "user", "content": user_input}
        ]
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=self.enable_thinking
        )
        return self.tokenizer(text, return_tensors="pt").to(self.model.device)
    
    def _parse_response(self, inputs, generated_ids):
        # Extract the output IDs from the generated IDs
        output_ids = generated_ids[0][len(inputs.input_ids[0]):].tolist()
        try:
            index = len(output_ids) - output_ids[::-1].index(151668)
        except ValueError:
            index = 0

        thinking_content = self.tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
        raw_response = self.tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
        response = self._parse_json(raw_response)
        print(f"Parsed response: {response}")
        return response, thinking_content
    
    def _parse_json(self, raw_response: str) -> list[dict[str,str]]:
        # Remove code block markers and leading/trailing whitespace
        cleaned = raw_response.strip()
        if cleaned.startswith("```json"):
            cleaned = cleaned[len("```json"):].strip()
        if cleaned.endswith("```"):
            cleaned = cleaned[:-3].strip()

        # Now parse as JSON
        try:
            return json.loads(cleaned)
        except json.JSONDecodeError as e:
            return []        
        


## Define the System prompt

In [198]:
# Define the one-shot reasoning and task prompt
# This prompt is designed to guide the model through a structured reasoning process
SYS_PROMPT = """
You are an advanced AI research assistant specialized in identifying and classifying datasets used within academic research papers.
Your primary goal is to accurately extract and categorize dataset identifiers (dataset_ids) from provided paper sections.

---

### Input Data Structure

You will receive a JSON string representing key sections of an academic paper, structured as follows:

```json
{
    "title": "Title of the paper",
    "author": "The primary author of the paper, e.g., 'Author A'",
    "abstract": "Abstract of the paper",
    "data_availability": "Data availability information",
    "other_dataset_citations": [
        {"dataset_ids": ["10.12345/12345"], "citation_context": "Dataset citation context 1"},
        {"dataset_ids": ["10.1234/xxxx.1x1x-xx11", "EPI_ISL_12345678"], "citation_context": "Dataset citation context 2"},
        ...
    ]
}
```

**Guidance on Input Sections:**
*   **`title`**: Provides general context for the paper's topic.
*   **`author`**: The primary author of the paper. This is crucial for determining if the dataset's *raw data* was *originally generated by this author*.
*   **`abstract`**: **CRITICAL** for understanding the research scope and, most importantly, for determining if a dataset's *raw data* was *originally generated by the authors of *this* paper*.
*   **`data_availability`**: This section provides information on datasets. Its content must be evaluated to determine if the data was *originally generated by the authors of this paper* (Primary) or *acquired from an existing source* (Secondary).
*   **`other_dataset_citations`**: A list of potential dataset citations. The `citation_context` is vital to confirm if a `dataset_id` truly refers to a dataset and to aid in classifying its origin (Primary or Secondary).

---

### Core Objective & Critical Exclusion

Your overarching objective is to identify and classify **only valid, data-related `dataset_id`s**.

**CRITICAL EXCLUSION**: You **MUST NOT** extract any `dataset_id`s that refer to other academic papers, articles, or the paper itself. Focus strictly on identifiers for *datasets*.

---

### Key Definitions

*   **`dataset_id`**: A unique, persistent identifier for a dataset. There are two main types:

    1.  **Digital Object Identifier (DOI)**:
        *   **Format**: `[prefix]/[suffix]`. The prefix always starts with "10." and is followed by a 4 or 5 digit number. The suffix can contain letters, numbers, and special characters.
        *   May or may not start with "https://doi.org/" or "doi:".
        *   **IMPORTANT DOI VALIDATION RULE**:
            *   Only identify DOIs that are explicitly used as `dataset_id`s.
            *   **DO NOT extract DOIs for academic papers/articles.**
            *   **If a DOI is presented as a reference to a publication (e.g., "as described in [DOI]", "cited in [DOI]", "see [DOI] for details on the method"), it is NOT a dataset_id.**
            *   A DOI is a `dataset_id` ONLY if the surrounding `citation_context` or `data_availability` section clearly indicates it refers to a dataset, data repository, data archive, or similar data-specific entity.

    2.  **Accession ID**:
        *   Typically alphanumeric strings that uniquely identify a dataset within a specific data repository.
        *   Can be found in both `data_availability` and `other_dataset_citations` sections.
        *   *Examples*: `"EPI_ISL_12345678"` (EPI dataset), `"IPR000264"` (InterPro dataset), `"SAMN07159041"` (NCBI Sequence Read Archive dataset), `"CHEMBL1782574"` (ChEMBL dataset)

*   **Dataset Type Classification**: **This is the MOST CRITICAL distinction. Focus ONLY on the *ORIGIN* of the *RAW DATA*.**

    *   **Primary**: Raw or processed data that was **ORIGINALLY GENERATED, COLLECTED, or CREATED by the AUTHORS OF *THIS SPECIFIC PAPER*** as part of the novel research presented in this paper. This data is central to the new findings and was *not* acquired from an existing public source.
        *   *Keywords often associated with Primary*: "generated in this study", "our data", "newly sequenced", "collected for this project", "created by us", "original data produced here", "developed by us".
        *   *Example*: If the paper describes *new* patient scans or *new* experimental results that *these authors* produced, and then provides a DOI for *those new scans/results*, that's Primary.

    *   **Secondary**: Raw or processed data that was **ACQUIRED, DERIVED, REUSED, or RE-ANALYZED from EXISTING, PUBLICLY AVAILABLE RECORDS or PREVIOUSLY PUBLISHED DATASETS** that were *not originally generated by the authors of this specific paper*. Even if the authors perform significant new processing, annotation, or analysis *on* this existing data, the *original raw data source* is considered Secondary if it was not their creation.
        *   *Keywords often associated with Secondary*: "publicly available data", "previously published", "existing", "external", "re-analyzed data", "obtained from", "data from", "acquired from", "derived from", "based on data from", "sourced from".
        *   **CRITICAL EXAMPLE FOR SECONDARY**: In the provided abstract, if it states "CT scans **acquired from The Cancer Imaging Archive 'NSCLC Radiomics' data collection**", then the `dataset_id` for "NSCLC Radiomics" (e.g., `10.7937/K9/TCIA.2015.PF0M9REI`) is **Secondary**. This is because the raw CT scans were *not* originally generated by the authors of *this* paper; they *used* this existing, external data.

---

### Classification Logic Flow (for each identified `dataset_id`):

To classify a `dataset_id` as Primary or Secondary, follow these steps strictly:

1.  **Examine the `abstract` and `citation_context` (if available) for the `dataset_id` in question.**
2.  **Look for explicit phrases indicating the *origin* of the *raw data*:**
    *   Does the text state the data was "acquired from", "obtained from", "derived from", "from [a named external archive/database]", "publicly available", or "previously published"?
        *   **IF YES**: Classify as **Secondary**. (e.g., `10.7937/K9/TCIA.2015.PF0M9REI` is Secondary because the abstract says "CT scans acquired from The Cancer Imaging Archive 'NSCLC Radiomics' data collection").
    *   Does the text state the data was "generated by us", "created by the authors", "our data", "newly sequenced", or describe a process of *original data collection/creation* by the authors of *this paper*?
        *   **IF YES**: Classify as **Primary**. (e.g., `10.7937/tcia.2020.6c7y-gq39` is Primary because the abstract says "This manuscript describes a dataset of thoracic cavity segmentations and discrete pleural effusion segmentations **we have annotated**" and describes the generation process by the authors).
3.  **If, after applying the above rules, the origin of the *raw data* remains truly ambiguous, then default to "Primary".**

---

### Tasks: Step-by-Step Instructions

Follow these three tasks in order:

**Task 1: Identify Valid Dataset IDs**

1.  **Search Priority**: Begin by searching the `data_availability` section. Then, search the `other_dataset_citations` section.
2.  **Validation**: For each potential `dataset_id` (DOI or Accession ID), confirm it is truly data-related.
    *   **For DOIs**: Strictly apply the **IMPORTANT DOI VALIDATION RULE** defined above. If it refers to a publication, **DO NOT** extract it.
    *   **For all IDs**: Look for surrounding terms like "data release", "data availability", "dataset", "database", "repository", "data source", "data access", or "data archive" within the `data_availability` section or the `citation_context`.
3.  **Deduplication**: If the same `dataset_id` is found multiple times, **only process the first instance encountered**.
4.  **Conditional Proceeding**:
    *   If **no valid `dataset_id`s are found** after searching both sections, **skip directly to Task 3** and output the "Missing" JSON structure.
    *   If one or more valid `dataset_id`s are found, proceed to Task 2.

**Task 2: Classify Dataset Types**

1.  For each valid `dataset_id` identified in Task 1, classify its type as either "Primary" or "Secondary".
2.  **STRICTLY APPLY THE "CLASSIFICATION LOGIC FLOW" ABOVE for each `dataset_id`.** Use the `author` section, the `abstract` section, and the `citation_context` to determine if the *raw data* associated with the `dataset_id` was *originally generated by the authors of *this* paper* (Primary) or *acquired/reused from an existing source* (Secondary).
3.  Apply the "Key Definitions" for Primary and Secondary types, paying close attention to the associated keywords and the provided examples.
4.  Remember the "Fallback Rule": Only default to "Primary" if, after careful consideration, the classification remains truly ambiguous regarding the *original generation* of the raw data.

**Task 3: Format and Return Results**

Return your final results as a JSON array of objects.

1.  **Scenario A: No Valid Datasets Found**
    If Task 1 resulted in no valid `dataset_id`s, return a single JSON object with the following structure:
    ```json
    [
        {
            "dataset_id": "Missing",
            "type": "Missing"
        }
    ]
    ```
2.  **Scenario B: One or More Valid Datasets Found**
    If Task 1 identified one or more valid `dataset_id`s, return every valid dataset found in a JSON array of objects, where each object has the following structure:
    ```json
    [
        {
            "dataset_id": "example_id_1",
            "type": "Primary"
        },
        {
            "dataset_id": "example_id_2",
            "type": "Secondary"
        },
        ...
    ]
    ```
"""

## Evaluate the articles

In [199]:
# Instantiate the QwenModelEval class with the model path and system prompt
inference_model = QwenModelEval(QWEN_BASE_MODEL_PATH, sys_prompt=SYS_PROMPT, enable_thinking=True, max_new_tokens=1576)

Loading Qwen model and tokenizer from: C:\Users\jim\.cache\kagglehub\models\qwen-lm\qwen-3\transformers\0.6b\1


In [200]:
def evaluate_articles(file_paths_df: pd.DataFrame, model) -> pd.DataFrame:
    results = []
    for i, row in file_paths_df.iterrows():
        article_id = row['article_id']
        dataset_info = row['dataset_info']
        pdf_text = row['pdf_text']
        xml_text = row['xml_text']
        text_type = "XML" if xml_text else "PDF"
        text_content = xml_text if xml_text else pdf_text
        response = ""
        thinking_content = ""

        # Prepare the user input for the model
        user_input = f"Text Content: {text_content}\n"
        
        if text_content:
            print(f"Processing article {i}/{len(file_paths_df)}: {article_id}, type: {text_type}")
            # Generate response from the model
            response, thinking_content = model.generate_response(user_input)

        results.append({
            'article_id': article_id,
            'dataset_info': dataset_info,
            'text_type': text_type,
            'llm_input': user_input,
            'llm_response': response,
            'llm_thinking_content': thinking_content
        })

    return pd.DataFrame(results).sort_values(by=["article_id"]).reset_index(drop=True)


In [201]:
# Load the file paths DataFrame from the CSV file
file_paths_df = pd.read_csv(os.path.join(BASE_OUTPUT_DIR, 'file_paths.csv'))
# Fill NaN values in the 'xml_type' and 'xml_text' columns with empty strings
file_paths_df['xml_file_path'] = file_paths_df['xml_file_path'].fillna('')
file_paths_df['xml_text'] = file_paths_df['xml_text'].fillna('')
# Display the first few rows of the file paths DataFrame
print(f"File paths DataFrame shape: {file_paths_df.shape}")
display(file_paths_df.head(3))


File paths DataFrame shape: (30, 8)


,article_id,pdf_file_path,xml_file_path,dataset_type,dataset_info,xml_type,pdf_text,xml_text
0,10.1002_2017jc013030,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test,[{'dataset_id': 'https://doi.org/10.17882/4938...,tei,"{""title"":"""",""author"":""Marie Barbieux"",""abstrac...","{""title"":""Assessing the variability in the rel..."
1,10.1002_anie.201916483,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test,"[{'dataset_id': 'Missing', 'type': 'Missing'}]",jats,"{""title"":"""",""author"":""Daniel Werner"",""abstract...","{""title"":""Effective and Reversible Carbon Diox..."
2,10.1002_anie.202005531,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test,"[{'dataset_id': 'Missing', 'type': 'Missing'}]",bioc,"{""title"":"""",""author"":""Dong Wang"",""abstract"":""A...","{""title"":""Trifluoromethyl Sulfoxides: Reagents..."


In [202]:
sample_file_paths_df = file_paths_df.copy().sample(3, random_state=42).reset_index(drop=True)
sample_file_paths_df

,article_id,pdf_file_path,xml_file_path,dataset_type,dataset_info,xml_type,pdf_text,xml_text
0,10.1002_mp.14424,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test,[{'dataset_id': 'https://doi.org/10.7937/k9/tc...,jats,"{""title"":"""",""author"":""Kendall J. Kisera"",""abst...","{""title"":""PleThora: Pleural effusion and thora..."
1,10.1002_ece3.6144,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test,[{'dataset_id': 'https://doi.org/10.5061/dryad...,jats,"{""title"":"""",""author"":""Beng"",""abstract"":""Ecolog...","{""title"":""Efficacy of metabarcoding for identi..."
2,10.1002_ejoc.202000139,./kaggle/input/make-data-count-finding-data-re...,,test,"[{'dataset_id': 'Missing', 'type': 'Missing'}]",unknown,"{""title"":"""",""author"":""Ann Christin Reiers\u00f...",


In [ ]:
processed_articles_df = evaluate_articles(file_paths_df, inference_model)
# Save processed_articles_df to CSV
processed_articles_df.to_csv("sample_evaluated_articles.csv", index=False)
print(f"Processed articles DataFrame shape: {processed_articles_df.shape}")
processed_articles_df

Processing article 0/3: 10.1002_mp.14424, type: XML
Preparing input with length: 2784
Parsed response: [{'dataset_id': '10.7937/tcia.2020.6c7y-gq39', 'type': 'Primary'}, {'dataset_id': '10.7937/K9/TCIA.2015.PF0M9REI', 'type': 'Secondary'}]
Processing article 1/3: 10.1002_ece3.6144, type: XML
Preparing input with length: 2184
Parsed response: [{'dataset_id': '10.5061/dryad.zw3r22854', 'type': 'Primary'}]
Processing article 2/3: 10.1002_ejoc.202000139, type: PDF
Preparing input with length: 1383
Parsed response: [{'dataset_id': '10.1002/ejoc.202000139', 'type': 'Primary'}, {'dataset_id': '10.1002/ejoc.202000139', 'type': 'Primary'}]


,article_id,dataset_info,text_type,llm_input,llm_response,llm_thinking_content
0,10.1002_ece3.6144,[{'dataset_id': 'https://doi.org/10.5061/dryad...,XML,"Text Content: {""title"":""Efficacy of metabarcod...","[{'dataset_id': '10.5061/dryad.zw3r22854', 'ty...","<think>\nOkay, let's tackle this problem. The ..."
1,10.1002_ejoc.202000139,"[{'dataset_id': 'Missing', 'type': 'Missing'}]",PDF,"Text Content: {""title"":"""",""author"":""Ann Christ...","[{'dataset_id': '10.1002/ejoc.202000139', 'typ...","<think>\nOkay, let's tackle this problem. The ..."
2,10.1002_mp.14424,[{'dataset_id': 'https://doi.org/10.7937/k9/tc...,XML,"Text Content: {""title"":""PleThora: Pleural effu...","[{'dataset_id': '10.7937/tcia.2020.6c7y-gq39',...","<think>\nOkay, let's tackle this problem step ..."


In [227]:
def format_dataset_id(dataset_id: str) -> str:
    """
    Formats the dataset_id by removing any leading/trailing whitespace and ensuring it is a string.
    
    Args:
        dataset_id (str): The dataset identifier to format.
        
    Returns:
        str: The formatted dataset identifier.
    """
    if dataset_id and dataset_id.startswith("10.") and len(dataset_id) > 10:
        # If the dataset_id starts with "10." and is longer than 10 characters, it's likely a DOI
        dataset_id = "https://doi.org/" + dataset_id.lower().strip()
    return dataset_id

# Create a DataFrame to hold the evaluation results by expaning the 'llm_response' column
def expand_evaluation_results(df: pd.DataFrame) -> pd.DataFrame:
    """
    Expands the evaluation results DataFrame by extracting dataset_id and type from the 'llm_response' column.
    
    Args:
        df (pd.DataFrame): The DataFrame containing evaluation results.
        
    Returns:
        pd.DataFrame: A new DataFrame with expanded dataset_id and type columns.
    """
    expanded_rows = []
    for _, row in df.iterrows():
        article_id = row['article_id']
        article_doi = article_id.replace('_', '/')
        datasets = row['llm_response']
        if datasets:
            for dataset in datasets:
                dataset_id = dataset.get('dataset_id', 'Missing')
                # Skip if the dataset_id is the same as the article DOI
                if dataset_id == article_doi:
                    # If the dataset_id is the same as the article DOI add it as Missing
                    expanded_rows.append({
                        'article_id': article_id,
                        'dataset_id': 'Missing',
                        'type': 'Missing',
                    })
                else:
                    expanded_rows.append({
                        'article_id': article_id,
                        'dataset_id': dataset.get('dataset_id', 'Missing'),
                        'type': dataset.get('type', 'Missing'),
                    })
    
    # Create a DataFrame from the expanded rows
    expanded_df = pd.DataFrame(expanded_rows)
    expanded_df['dataset_id'] = expanded_df['dataset_id'].apply(format_dataset_id)  # Format dataset_id
    expanded_df['type'] = expanded_df['type'].str.strip().str.capitalize()  # Ensure type is capitalized and stripped of whitespace
    expanded_df = expanded_df.sort_values(by=["article_id", "dataset_id", "type"], ascending=False).drop_duplicates(subset=['article_id', 'dataset_id'], keep="first").reset_index(drop=True)
    
    return expanded_df


In [229]:
sample_submission_df = expand_evaluation_results(processed_articles_df)
sample_submission_df.to_csv(os.path.join(BASE_OUTPUT_DIR, 'sample_submission.csv'), index=False)
print(f"Submission DataFrame shape: {sample_submission_df.shape}")
display(sample_submission_df.head(10))

Submission DataFrame shape: (4, 3)


,article_id,dataset_id,type
0,10.1002_mp.14424,https://doi.org/10.7937/tcia.2020.6c7y-gq39,Primary
1,10.1002_mp.14424,https://doi.org/10.7937/k9/tcia.2015.pf0m9rei,Secondary
2,10.1002_ejoc.202000139,Missing,Missing
3,10.1002_ece3.6144,https://doi.org/10.5061/dryad.zw3r22854,Primary
